In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import os
import itertools
import random
import sys
import powerlaw as pl
sys.path.append('/Users/css/dev/thesis/selfish_mining_abm/selfish_mining_abm')
import blockchain

# config

In [4]:
# PARAMETERS TO LOOP OVER
alphas = np.linspace(0, 0.5, 3)
alpha = 0.3
# 1 minute equals 60'000 milliseconds.
gammas = np.linspace(100, 500, 1) / 60000
gamma = 0.1
# number of repetitions to average results over
repetitions = 1

# Hand/pick selfish miners: "RANDOM", "BETWEENNESS"
# centrality_measures = ["RANDOM", "BETWEENNESS"]
centrality_measure = "RANDOM"

# available topologogies: "UNIFORM", "ER", "BA"
# topologies = ["UNIFORM", "ER", "BA"]
topology = "UNIFORM"

# available hashing power distributions: "UNIFORM", "POWERLAW", "EXPONENTIAL"
# hash_distributions = ["UNIFORM", "POWERLAW", "EXPONENTIAL"]
hash_distribution = "UNIFORM"

# SPECFIY TOPOLOGY
desired_avg_degree = 5  # applies to ER and RAND topology.
ba_m = 5  # relevant for BA topology; no. edges to attach from new node to existing nodes

# SPECIFY HASHING POWER DISTRIBUTION
pl_alpha = 1.88  # input parameter for powerlaw distribution
exp_lambda = 1  # input parameter for exponential distribution
# ADDITIONAL PARAMETERS
simulation_time = 1000
number_of_nodes = 100
number_selfish_nodes = 1  # if there is more than 1 selfish miner, they act as "cartel"
number_honest_nodes = number_of_nodes - number_selfish_nodes
verbose = False

# Set up

In [5]:
def set_up_topology(topology, number_of_nodes, desired_avg_degree, ba_m):
    # generate network depending on topology parameter
    if topology == "UNIFORM":
        net_p2p = nx.random_degree_sequence_graph(
            [desired_avg_degree for i in range(number_of_nodes)])

    elif topology == "ER":
        p = desired_avg_degree / number_of_nodes
        net_p2p = nx.fast_gnp_random_graph(number_of_nodes, p)

    elif topology == "BA":
        net_p2p = nx.barabasi_albert_graph(number_of_nodes, ba_m)

    # get largest connected component
    lcc_set = max(nx.connected_components(net_p2p), key=len)
    net_p2p = net_p2p.subgraph(lcc_set).copy()
    # some nodes may have been removed because they were not port of the lcc.
    # relabel nodes so that only nodes in lcc are labelled. (without it we run into problems where node labels are higher than the number of nodes -> loops run into indexing problems)
    net_p2p = nx.convert_node_labels_to_integers(
        net_p2p, first_label=0)

    # some nodes may have been removed as they were not part of the lcc -> update num nodes
    number_of_nodes = len(net_p2p)
    # (brute-forcing number of selfish nodes to stay unchanged)
    number_honest_nodes = number_of_nodes - number_selfish_nodes

    return net_p2p, number_honest_nodes

def set_up_hash_distr(net_p2p, centrality_measure, hash_distribution, number_selfish_nodes, number_honest_nodes, alpha):
    # make sure that when there are no selfish nodes that alpha is never unequal 0. (in case you want to simulate only honest nodes)
    assert not (number_selfish_nodes == 0 and alpha !=
                0), "Alpha unequal 0 with no selfish nodes"

    if hash_distribution == "UNIFORM":
        hashing_power_selfish = np.random.random(number_selfish_nodes)
        hashing_power_honest = np.random.random(number_honest_nodes)

    elif hash_distribution == "POWERLAW":
        power_distrib = pl.Power_Law(parameters=[pl_alpha], discrete=False)
        hashing_power_selfish = power_distrib.generate_random(
            number_selfish_nodes)
        hashing_power_honest = power_distrib.generate_random(
            number_honest_nodes)

    elif hash_distribution == "EXPONENTIAL":
        exp_distrib = pl.Exponential(parameters=[exp_lambda])
        hashing_power_selfish = exp_distrib.generate_random(
            number_selfish_nodes)
        hashing_power_honest = exp_distrib.generate_random(
            number_honest_nodes)

    # normalize vector so that sum of selfish hashing power equals alpha & honest hashing power equals 1-alpha.
    if number_selfish_nodes != 0:
        hashing_power_selfish /= sum(hashing_power_selfish)
        hashing_power_selfish *= alpha
    hashing_power_honest /= sum(hashing_power_honest) / (1 - alpha)

    # combine selfish and honest hashing power vectors together
    hashing_power_unsorted = np.append(
        hashing_power_selfish, hashing_power_honest)

    if centrality_measure == "RANDOM":
        # create an is_selfish vector that corresponds to the order of the hashing_power vector
        is_selfish = np.append(np.ones(number_selfish_nodes),
                                np.zeros(number_honest_nodes))

        # finally, randomize is_selfish and hashing_power arrays in unison
        randomize = np.arange(len(hashing_power_unsorted))
        np.random.shuffle(randomize)
        hashing_power = hashing_power_unsorted[randomize]
        is_selfish = is_selfish[randomize]

    elif centrality_measure == "BETWEENNESS":
        # compute betweenness centrality and sort it
        btwn = nx.betweenness_centrality(net_p2p)
        btwn_sorted = {k: v for k, v in sorted(
            btwn.items(), key=lambda item: item[1], reverse=True)}
        # return node indeces sorted for betweenness centrality
        btwn_sorted_indices = list(btwn_sorted.keys())

        selfish_indices = list(btwn_sorted.keys())[:number_selfish_nodes]
        honest_indices = list(btwn_sorted.keys())[
            number_selfish_nodes:len(btwn)]

        # set selifsh nodes according to betweenness centrality
        is_selfish = np.zeros(number_honest_nodes+number_selfish_nodes)
        for i in selfish_indices:
            is_selfish[i] = 1

        # sort hashing power vector so that selfish nodes are assigned correct hashing power
        hashing_power = hashing_power_unsorted.copy()
        for (index, value) in enumerate(btwn_sorted):
            hashing_power[value] = hashing_power_unsorted[index]

    return hashing_power, is_selfish

def set_up_model(
    centrality_measure, topology, hash_distribution, number_of_nodes, number_selfish_nodes, alpha, desired_avg_degree, ba_m
):
    net_p2p, number_honest_nodes = set_up_topology(
        topology, number_of_nodes, desired_avg_degree, ba_m)

    hashing_power, is_selfish = set_up_hash_distr(
        net_p2p, centrality_measure, hash_distribution, number_selfish_nodes, number_honest_nodes, alpha)

    return net_p2p, hashing_power, is_selfish

In [6]:
def simulate(centrality_measure, topology, hash_distribution, number_of_nodes, number_selfish_nodes, alpha, desired_avg_degree, ba_m):
    if alpha == 0:
        net_p2p, hashing_power, is_selfish = set_up_model(
            centrality_measure, topology, hash_distribution, number_of_nodes, number_selfish_nodes, 0, desired_avg_degree, ba_m
        )
    else:
        net_p2p, hashing_power, is_selfish = set_up_model(
            centrality_measure, topology, hash_distribution, number_of_nodes, number_selfish_nodes, alpha, desired_avg_degree, ba_m
        )

    model = blockchain.GillespieBlockchain(
        net_p2p, is_selfish, hashing_power, gamma, verbose=verbose
    )

    while model.time < simulation_time:
        model.next_event()
    model.block_tree.tag_main_chain()
    
    return model

# Simulation

In [7]:
model = simulate(centrality_measure, topology, hash_distribution, number_of_nodes, number_selfish_nodes, alpha, desired_avg_degree, ba_m)

# Playground
### You can play around and access everything from above simulation

In [8]:
model.block_tree.max_height

63

### MSB

In [9]:
# create list of miner ID's for all mainchain blocks
mc_miner_id_list = []

for block in model.block_tree.tree.nodes():
    if model.block_tree.attributes[block]["main_chain"]:
        mc_miner_id_list.append(model.block_tree.attributes[block]["miner"])

# remove genesis block
mc_miner_id_list.pop(0)

# Compute C_i value for honest miners
C_i = 0
# iterating over mc_miner_id_list without the last value so that I don't run into indexing issues.
for (index, value) in enumerate(mc_miner_id_list[:-1]):
    # first check: that miner mined two consecutive blocks | second check: that miner is honest
    if mc_miner_id_list[index] == mc_miner_id_list[index+1] and model.is_selfish[value] == False:
        C_i += 1

# Compute S_i value
# shuffle chain
repititions = 100
S_i_list = []
for rep in range(repititions):
    # shuffle chain
    shuffled_mc_miner_id_list = mc_miner_id_list.copy()
    random.shuffle(shuffled_mc_miner_id_list)
    # compute average S_i value for each honest miner
    C_i_rnd = 0

    for (index, value) in enumerate(shuffled_mc_miner_id_list[:-1]):
        # first check: that miner mined two consecutive blocks | second check: that miner is honest
        if (shuffled_mc_miner_id_list[index] == shuffled_mc_miner_id_list[index+1]) and (model.is_selfish[value] == False):
            C_i_rnd += 1
    S_i_list.append(C_i_rnd)

avg_S_i = np.mean(S_i_list)
std_S_i = np.std(S_i_list)

if std_S_i != 0:
    msb_i = (C_i - avg_S_i) / std_S_i
else:
    msb_i = C_i - avg_S_i

In [11]:
print(C_i)
print(S_i_list)
print(avg_S_i)
print(msb_i)

1
[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
0.26
1.4729467430173613


In [12]:
print(model.is_selfish)
print(mc_miner_id_list)
print(shuffled_mc_miner_id_list)
print(S_i_list)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[70, 17, 14, 78, 79, 15, 15, 15, 33, 3, 6, 2, 57, 9, 95, 94, 71, 13, 4, 26, 52, 46, 11, 43, 47, 86, 4, 21, 15, 15, 15, 67, 95, 13, 77, 56, 15, 15, 75, 20, 52, 13, 30, 87, 15, 15, 15, 53, 97, 97, 22, 86, 14, 20, 15, 15, 15, 15, 15, 7, 6, 34, 15]
[56, 6, 77, 15, 15, 53, 3, 22, 52, 6, 14, 15, 7, 20, 15, 33, 15, 21, 15, 70, 15, 15, 52, 95, 87, 9, 15, 15, 97, 15, 15, 67, 15, 75, 43, 20, 11, 15, 47, 14, 71, 30, 94, 95, 26, 86, 79, 34, 13, 17, 57, 15, 4, 15, 78, 4, 2, 13, 15, 46, 97, 13, 86]
[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,

### hashing power centrality

In [13]:
# Parameters needed
centrality_measure = "BETWEENNESS"
hash_distribution = "UNIFORM"
number_honest_nodes = 9
number_selfish_nodes = 1
net_p2p = nx.erdos_renyi_graph(number_selfish_nodes+number_honest_nodes, 0.1)
alpha = 0.1

In [14]:
# make sure that when there are no selfish nodes that alpha is never unequal 0. (in case you want to simulate only honest nodes)
assert not (number_selfish_nodes == 0 and alpha !=
            0), "Alpha unequal 0 with no selfish nodes"

if hash_distribution == "UNIFORM":
    hashing_power_selfish = np.random.random(number_selfish_nodes)
    hashing_power_honest = np.random.random(number_honest_nodes)

# normalize vector so that sum of selfish hashing power equals alpha & honest hashing power equals 1-alpha.
if number_selfish_nodes != 0:
    hashing_power_selfish /= sum(hashing_power_selfish)
    hashing_power_selfish *= alpha
hashing_power_honest /= sum(hashing_power_honest) / (1 - alpha)

# combine selfish and honest hashing power vectors together
hashing_power_unsorted = np.append(hashing_power_selfish, hashing_power_honest)

hashing_power_sorted = np.append(
    sorted(hashing_power_selfish, reverse=True),
    sorted(hashing_power_honest, reverse=True)
)

### "BETWEENNESS":
# compute betweenness centrality and sort it
btwn = nx.betweenness_centrality(net_p2p)
btwn_sorted = {k: v for k, v in sorted(
    btwn.items(), key=lambda item: item[1], reverse=True)}
# return node indeces sorted for betweenness centrality
btwn_sorted_indices = list(btwn_sorted.keys())

selfish_indices = list(btwn_sorted.keys())[:number_selfish_nodes]
# honest_indices = list(btwn_sorted.keys())[
#     number_selfish_nodes:len(btwn)]

# set selifsh nodes according to betweenness centrality
is_selfish = np.zeros(number_honest_nodes+number_selfish_nodes)
for i in selfish_indices:
    is_selfish[i] = 1

# sort hashing power vector so that selfish nodes are assigned correct hashing power
hashing_power_btwn = hashing_power_unsorted.copy()
for (index, value) in enumerate(btwn_sorted_indices):
    hashing_power_btwn[value] = hashing_power_unsorted[index]

### "HASHINGPOWER":
# compute betweenness centrality and sort it
# sort hashing power vector so that selfish nodes are assigned correct hashing power
hashing_power_hash = hashing_power_unsorted.copy()
for (index, value) in enumerate(btwn_sorted_indices):
    hashing_power_hash[value] = hashing_power_sorted[index]

In [15]:
print(list(btwn_sorted.keys()))
print(list(btwn_sorted.keys())[:number_selfish_nodes])

[7, 0, 1, 2, 3, 4, 5, 6, 8, 9]
[7]


### chainsplit stats

In [1]:
def get_chainsplit_stats():
    # get mainchain and orphan block IDs
    all_blocks = []
    mc_blocks = []
    orphan_blocks = []

    for block in model.block_tree.tree.nodes():
        if model.block_tree.attributes[block]["main_chain"]:
            mc_blocks.append(model.block_tree.attributes[block]["id"])
        else:
            orphan_blocks.append(model.block_tree.attributes[block]["id"])
        all_blocks.append(model.block_tree[block]["id"])
    # disregard genesis block
    mc_blocks.pop(0)
    all_blocks.pop(0)
            
    # COUNT NUMBER OF MAINCHAIN SPLITS
    # count number of main chain splits (disregarding genesis block)
    mainchain_split_count = 0
    for orphan in orphan_blocks:
        parent_block = list(model.block_tree.tree.predecessors(orphan))[0]
        if model.block_tree.attributes[parent_block]["main_chain"] and not model.block_tree[parent_block]["miner"] == "genesis":
            mainchain_split_count += 1


    # COUT NUMBER OF ALL CHAIN SPLITS (ORPHAN & MAINCHAIN)
    # count number of chain splits (disregarding genesis block)
    chain_split_count = 0
    for block in all_blocks:
        num_children = len(list(model.block_tree.tree.successors(block)))
        if num_children > 1:
            chain_split_count += (num_children - 1)

    # COMPUTE PROBABILITY OF MAIN CHAIN BLOCK HAVING NO SIBLINGS       
    # count blocks on main chain that have more than 1 child (disregarding genesis block)
    mainchain_no_sibling_count = 0
    for block in mc_blocks:
        if not len(list(model.block_tree.tree.successors(block))) > 1:
            no_sibling_count += 1
    return mainchain_split_count, chain_split_count, mainchain_no_sibling_count